# Probit Regressions

---
## Experimental Setup

### Libraries & Settings

In [1]:
import os              # General OS commands
import numpy as np     # NumPy
import pandas as pd    # Python Data Analysis Library
import zipfile         # Compress/decompress ZIP files
import sqlite3         # SQLite3 Database Driver
import re              # Regular Expressions

In [2]:
import statsmodels.api as sm
from sklearn import metrics
import math

In [3]:
# Never truncate columns, display all the data
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', -1)

# Display floating-point numbers with 4 decimals in `pandas.DataFrame`
pd.options.display.float_format = '{:,.4f}'.format

import matplotlib.pyplot as plt
# Display MatPlotLib stuff inline
%matplotlib inline

### Database

In [4]:
zip_filename = "../../data/ee-insee-2015_custom-sqlite.zip"
eedb = zip_filename.replace("-sqlite.zip", ".sqlite")

if not os.path.exists(eedb):
    with zipfile.ZipFile(zip_filename) as zip_file:
        zip_file.extractall("../../data/")

In [5]:
with sqlite3.connect(eedb) as con:
    query = "SELECT * FROM eec15_custom"
    eec15 = pd.read_sql_query(query, con)

In [6]:
# Create a ("female" x "enfant") interaction variable
bool_ = eec15[["enfants_", "female_"]].astype(bool)
eec15["female_enfants_"] = (bool_.enfants_ & bool_.female_).astype(int)

# Drop data we don't need
eec15 = eec15[eec15.age60_ == 0]
eec15 = eec15.drop("age60_", 1)

---
## Regression

In [7]:
# Build a list of parameters to include in the model, using regex
# https://www.datarobot.com/blog/multiple-regression-using-statsmodels/
filters = {
    "age": "^age[0-9]{2}_$",
    "diploma": "^dip[0-9]{2}_$",
    "etranger": "^etranger_$",
    "domtom": "^domtom_$",
#     "trim": "^trim$",
    "female": "^female_$",
    "enfants": "^enfants_$",
#     "female_enfants": "^female_enfants_$",
    "region": "^region[1-2]_$"
}
params = {k: sorted([x for x in eec15.columns if re.match(r, x)]) for (k, r) in filters.items()}

# Avoid the dummy variable trap
params = {k: (v if len(v) == 1 else v[:-1]) for (k, v) in params.items()}
params["region"] += ["region2_"]

In [8]:
trims = ["t{}".format(x) for x in sorted(eec15["trim"].unique())]
eec15_ = {t: eec15[eec15.trim == int(t[1])] for t in trims}
X = {t: eec15_[t][sum(params.values(), [])] for t in eec15_}
y = {t: eec15_[t]["actop_"] for t in eec15_}

### Train the model using `trim=1` data

In [9]:
# Fit using `statsmodels`
reg_probit_sm = sm.Probit(y["t1"], sm.add_constant(X["t1"])).fit(disp=False)

# Print the (`statsmodels`) model summary
reg_probit_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                 actop_   No. Observations:                72838
Model:                         Probit   Df Residuals:                    72818
Method:                           MLE   Df Model:                           19
Date:                Thu, 16 Feb 2017   Pseudo R-squ.:                  0.2213
Time:                        22:03:28   Log-Likelihood:                -36790.
converged:                       True   LL-Null:                       -47245.
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -0.4084      0.024    -17.055      0.000        -0.455    -0.361
etranger_      0.4105      0.020     20.054      0.000         0.370     0.451
age15_         0.9455      0.015     63.398      0.000         0.916     0.975
age30_         0.1671      0.017      9.953      0.000         0.134     0.200
age40_        -0.0488      0.016     -3.057      0.002        -0.080    -0.018
dip10_        -1.1090      0.021    -52.166      0.000        -1.151    -1.067
dip11_        -1.3303      0.038    -35.100      0.000        -1.405    -1.256
dip30_        -0.6348      0.055    -11.518      0.000        -0.743    -0.527
dip31_        -1.0496      0.023    -44.685      0.000        -1.096    -1.004
dip33_        -1.2131      0.043    -28.299      0.000        -1.297    -1.129
dip41_        -0.3054      0.023    -13.380      0.000        -0.350    -0.261
dip42_        -0.8262      0.020    -41.726      0.000        -0.865    -0.787
dip50_        -0.5785      0.017    -34.620      0.000        -0.611    -0.546
dip60_         0.1148      0.021      5.483      0.000         0.074     0.156
dip70_        -0.1178      0.042     -2.811      0.005        -0.200    -0.036
female_        0.3049      0.011     27.981      0.000         0.284     0.326
enfants_      -0.2666      0.012    -21.358      0.000        -0.291    -0.242
region1_       0.1179      0.019      6.225      0.000         0.081     0.155
region2_       0.1536      0.026      5.871      0.000         0.102     0.205
domtom_        0.3553      0.016     21.723      0.000         0.323     0.387
==============================================================================
"""

In [10]:
# Create a DataFrame of coefficients
regressors = list(X["t1"].columns) + ["const"]
coeffs = pd.DataFrame(index=regressors)
coeffs["sm"] = reg_probit_sm.params

# Note: in the Logit notebook, the purpose of this DataFrame was to compare the results
# from the `sklearn` and `statsmodels` regressions – we cannot do that here since `sklearn`
# cannot perform Probit regressions – we keep the `coeffs` variable for consistency

### Apply the model to test data (`trim=2/3/4`)

In [11]:
# To avoid the pitfall from the above calculation, we need to map [0, 1] -> {0, 1}
# We define a threshold probability above which we consider an individual to be employed
THRESHOLD = 0.5

for t in trims:
    predicted = np.where(reg_probit_sm.predict(sm.add_constant(X[t])) < THRESHOLD, 0, 1)
    print "Accuracy ({}): {}".format(t, metrics.accuracy_score(y[t], predicted))

Accuracy (t1): 0.761745242868
Accuracy (t2): 0.766766416825
Accuracy (t3): 0.759427790539
Accuracy (t4): 0.762881767454


In [12]:
for t in trims:
    predicted_proba = reg_probit_sm.predict(sm.add_constant(X[t]))
    print "ROC Area-Under-Curve ({}): {}".format(t, metrics.roc_auc_score(y[t], predicted_proba))

ROC Area-Under-Curve (t1): 0.799250365035
ROC Area-Under-Curve (t2): 0.802042228518
ROC Area-Under-Curve (t3): 0.796118251953
ROC Area-Under-Curve (t4): 0.80215302874


### Accuracy of prediction of `1` and `0`

In [13]:
for actop_ in [0, 1]:
    print "===== actop_ = {} =====".format(actop_)
    for t in trims:
        y_zero = y[t][y[t] == actop_]
        X_zero = X[t][X[t].index.map(lambda x: x in y_zero)]
    
        predicted = np.where(reg_probit_sm.predict(sm.add_constant(X_zero)) < THRESHOLD, 0, 1)
        print "Accuracy ({}): {}".format(t, metrics.accuracy_score(y_zero, predicted))
    print

===== actop_ = 0 =====
Accuracy (t1): 0.871154986866
Accuracy (t2): 0.87201814253
Accuracy (t3): 0.87058328349
Accuracy (t4): 0.874132416614

===== actop_ = 1 =====
Accuracy (t1): 0.560271514395
Accuracy (t2): 0.568164385774
Accuracy (t3): 0.547219285261
Accuracy (t4): 0.553907144025



---
## Marginal Effects

In [13]:
marginal_effects = pd.DataFrame()

### Analytically (using the derivative formula)
Recall that:
$$EM(x_1) = \frac{\partial{\mathbb{E}[y|x]}}{\partial{x_1}} = \beta_1 \Phi'(\beta_0 + \beta_1 x_1 + ... ) = \beta_1 \phi(\beta_0 + \beta_1 x_1 + ... )$$
where
$$\Phi(x^T\beta) = P(y=1|x) = \mathbb{E}[y|x]$$
is the *cumulative distribution function* and
$$\Phi'(x) = \phi(x) = \frac{1}{\sqrt{2\pi}}\, e^{-\frac{x^2}{2}}$$
the *probability density function* of a **standard** normal distribution ($\mu = 0, \sigma = 1$)

**Note**: for each parameter, we calculate the *mean* marginal effect over the entire training dataset

In [14]:
def phi(x):
    return 1/math.sqrt(2*math.pi) * math.exp(-x**2/2)

In [15]:
y_fitted = X["t1"].dot(coeffs["sm"][:-1])
y_fitted += coeffs["sm"]["const"]
dlambda_y = y_fitted.map(phi)

In [16]:
marginal_effects["analytical"] = coeffs["sm"]*dlambda_y.mean()

### By "rule-of-thumb"
Note (from above) that:
$$EM(x_1) = \beta_1 \phi(\beta_0 + \beta_1 x_1 + ... )$$
and 
$$\max \phi(x) = \phi(0) = \frac{1}{\sqrt{2\pi}} \approx \frac{1}{2.5}$$
Therefore, we have that:
$$EM(x_1) \approx \frac{\beta_1}{2.5}$$

The "rule-of-thumb" thus divides all regression coefficients by 2.5 to approximate the marginal effect.

In [17]:
marginal_effects["rule_of_thumb"] = coeffs["sm"]/2.5

### By modifying the dataset ("passage de tout le monde en licence")

In [18]:
brute_force = pd.Series()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1

#         proba_one = reg_logit_sk.predict_proba(X_one).T[1].mean()
#         proba_t1 = reg_logit_sk.predict_proba(X["t1"]).T[1].mean()
        proba_one = reg_logit_sm.predict(sm.add_constant(X_one, has_constant="add")).mean()
        proba_t1 = reg_logit_sm.predict(sm.add_constant(X["t1"])).mean()
        brute_force[pivot] = proba_one - proba_t1

In [19]:
marginal_effects["brute_force"] = brute_force
marginal_effects.transpose()

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_,const
analytical,0.1169,0.2693,0.0476,-0.0139,-0.3158,-0.3788,-0.1808,-0.2989,-0.3455,-0.0870,-0.2353,-0.1648,0.0327,-0.0335,0.0868,-0.0759,0.0336,0.0438,0.1012,-0.1163
rule_of_thumb,0.1642,0.3782,0.0668,-0.0195,-0.4436,-0.5321,-0.2539,-0.4198,-0.4852,-0.1221,-0.3305,-0.2314,0.0459,-0.0471,0.1220,-0.1066,0.0472,0.0615,0.1421,-0.1634
brute_force,0.1141,0.2080,-0.0517,-0.1133,-0.1650,-0.2120,-0.0360,-0.1509,-0.1882,0.0715,-0.0924,-0.0185,0.2174,0.1364,0.0421,-0.0435,0.0022,0.0125,0.0929,nan


In [20]:
ratios = marginal_effects.copy()
ratios = ratios.div(ratios["analytical"], axis=0)
ratios.transpose()

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_,const
analytical,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
rule_of_thumb,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046,1.4046
brute_force,0.9760,0.7724,-1.0862,8.1489,0.5225,0.5597,0.1992,0.5049,0.5449,-0.8226,0.3927,0.1122,6.6504,-4.0648,0.4854,0.5728,0.0654,0.2847,0.9185,nan
